<a href="https://colab.research.google.com/github/merucode/DL/blob/91-Colab-Kaggle-ML-Regression/01-01_%5BRegression-LR%5D_Bike-Sharing-Demand(basic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imformation

* Title : [Bike Sharing Demand](https://www.kaggle.com/competitions/bike-sharing-demand/overview)
* Type : Regression
* Evaluation : RMSLE
* Model : Random Forest Regression
* Python version: 3.10.6
* Basic library version
  * sklearn(scikit-learn==1.2.2)
  * numpy(numpy==1.22.4)
  * pandas(pandas==1.5.3)
  * matplotlib(matplotlib==3.7.1)
  * seaborn(seaborn==0.12.2)
  * datetime, calendar
* Addtional Library version
* Considering Library version
* Improvement

# STEP 0. Version check and Install Dependency

Step 0-1. Install Dependency

Step 0-2. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

In [ ]:
!pip list

Step 0-3. Download Data

In [ ]:
!export KAGGLE_USERNAME=merucode && export KAGGLE_KEY=4efadc2ddd497ca365f948c01bd11ead && kaggle competitions download -c bike-sharing-demand

In [ ]:
from zipfile import ZipFile

data_path = '/content/'

with ZipFile(data_path + 'bike-sharing-demand.zip') as zipper:
  zipper.extractall()

# STEP 1. Check Data
★ 분석결과
* (FE) datetime: 연도, 월, 시간 정보 추출 후 제거
* causal, registered: test 데이터셋에 없으므로 불필요

Step 1-1. Check data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content//'

# 훈련, 검증, 테스트 데이터 경로 설정
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
submission.head(3)

In [ ]:
train.info()

Step 1-2. Features Info


* (FE) datetime: 기록일시
* season: 계절(1: 봄, 2: 여름, 3: 가을, 4: 겨울)
* holiday: 공휴일 여부(0: 공휴일 아님, 1: 공휴일)
* workingday: 근무일 여부
* weather:날씨
* temp: 실제 온도
* atemp: 체감 온도
* humidity :상대 습도
* windspeed : 풍속
* (X) casual : 등록x 사용자 수
* (X) registered : 등록 사용자 수
* (Traget) count: 자전거 대여 수량

★ 분석결과
* (FE) datetime: 연도, 월, 시간 정보 추출 후 제거
* causal, registered: test 데이터셋에 없으므로 불필요


Step 1-3. Feature Engineering

In [ ]:
# 날짜 연도 월 일 나누기
print(train['datetime'][100].split()[0]) # 날짜
print(train['datetime'][100].split()[0].split("-")[0])  # 연도
print(train['datetime'][100].split()[0].split("-")[1])  # 월
print(train['datetime'][100].split()[0].split("-")[2])  # 일

# 시간 시, 분, 초 나누기
print(train['datetime'][100].split()[1]) # 시간
print(train['datetime'][100].split()[1].split(":")[0]) # 시
print(train['datetime'][100].split()[1].split(":")[1]) # 분
print(train['datetime'][100].split()[1].split(":")[2]) # 초

In [ ]:
# 날짜 피처 생성
train['date'] = train['datetime'].apply(lambda x: x.split()[0])  # 날짜 피처 생성

# 연도, 월, 일, 시, 분, 초 피처 생성
train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])
train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

In [ ]:
train.head(2)

In [ ]:
# 요일 피처 생성 설명
from datetime import datetime
import calendar

print(train['date'][100]) # 날짜
print(datetime.strptime(train['date'][100], '%Y-%m-%d'))  # datetime 타입 변경
print(datetime.strptime(train['date'][100], '%Y-%m-%d').weekday())  # 정수로 요일 반환
print(calendar.day_name[datetime.strptime(train['date'][100], '%Y-%m-%d').weekday()])  # 문자열로 요일 반환

In [ ]:
# 요일 피처 생성
train['weekday'] = train['date'].apply(
    lambda dateString: calendar.day_name[datetime.strptime(train['date'][100], '%Y-%m-%d').weekday()])

In [ ]:
train.head(2)

In [ ]:
# season, weather 피처 mapping(의미 파악을 위한 문자열 변환)
train['season'] = train['season'].map({1: 'Spring',
                                       2: 'Summer',
                                       3: 'Fall',
                                       4: 'Winter'})
train['weather'] = train['weather'].map({1: 'Clear',
                                         2: 'Mist, Few clouds',
                                         3: 'Light Snow, Rain, Thunderstorm',
                                         4: 'Heavy Rain, Thunderstorm, Snow, Fog'})

In [ ]:
train.head(2)

# STEP 2. Data Visualize
★ 분석결과
* [Traget] count: 비대칭 분포로 학습 수행시 log 변환 수행(회귀모델 정규분포일수록 성능 좋음), 마지막에는 지수 변환하여 실제 타깃값 복원 필요
* weather : weater==4 이상치 발생 → 이상치 제거
* windspeed : 결측값 많음, count와 상관관계 없음 → 제거
* day, minute, seconde : 분별력 없음 → 제거

STEP 2-1. Dis plot

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### DIS PLOT : 수치형 데이터 분포
mpl.rc('font', size=15) # 폰트 크기를 15로 설정

sns.displot(train['count']) # ★ 타깃값 count 비대칭 > log 변환 필요

In [ ]:
 sns.displot(np.log(train['count']))

STEP 2-2. Bar Plot

In [ ]:
### BAR PLOT : 범주형 데이터에 따른 수치형 데이터 정보
# m행 n열 Figure 준비
mpl.rc('font', size=14)      # 폰트 크기 설정
mpl.rc('axes', titlesize=15)  # 각 축 제목 크기 설정
figure, axes = plt.subplots(nrows=3, ncols=2) # 3행 2열 Figure 생성
plt.tight_layout()             # 그래프 사이 여백 확보
figure.set_size_inches(10, 9) # 전체 Figure 크기를 10*9인치로 설정

# 각 서브플롯 할당
sns.barplot(x='year', y='count', data=train, ax=axes[0, 0])
sns.barplot(x='month', y='count', data=train, ax=axes[0, 1])
sns.barplot(x='day', y='count', data=train, ax=axes[1, 0])
sns.barplot(x='hour', y='count', data=train, ax=axes[1, 1])
sns.barplot(x='minute', y='count', data=train, ax=axes[2, 0])
sns.barplot(x='second', y='count', data=train, ax=axes[2, 1])

# 세부설정
axes[0, 0].set(title='Rental amounts by year')
axes[0, 1].set(title='Rental amounts by month')
axes[1, 0].set(title='Rental amounts by day')     # ★ 분별력 없음 → 제거
axes[1, 1].set(title='Rental amounts by hour')
axes[2, 0].set(title='Rental amounts by minute')  # ★ 분별력 없음 → 제거
axes[2, 1].set(title='Rental amounts by second')  # ★ 분별력 없음 → 제거

# 1행 위치한 서브플롯들의 x축 라벨 90도 회전
axes[1, 0].tick_params(axis='x', labelrotation=90)
axes[1, 1].tick_params(axis='x', labelrotation=90)

Step 2-3. Box Plot

In [ ]:
### BOX PLOT : 범주형 데이터에 따른 수치형 데이터 정보
# m행 n열 Figure 준비
figure, axes = plt.subplots(nrows=2, ncols=2) # 2행 2열 Figure 생성
plt.tight_layout()             # 그래프 사이 여백 확보
figure.set_size_inches(10, 10) # 전체 Figure 크기를 10*10인치로 설정

# 각 서브플롯 할당
sns.boxplot(x='season', y='count', data=train, ax=axes[0, 0])
sns.boxplot(x='weather', y='count', data=train, ax=axes[0, 1])
sns.boxplot(x='holiday', y='count', data=train, ax=axes[1, 0])
sns.boxplot(x='workingday', y='count', data=train, ax=axes[1, 1])

# 세부설정
axes[0, 0].set(title='Box Plot On Count Across Season')
axes[0, 1].set(title='Box Plot On Count Across Weather')
axes[1, 0].set(title='Box Plot On Count Across Holiday')
axes[1, 1].set(title='Box Plot On Count Across Working Day')

# x축 라벨 겹침 해결
axes[0, 1].tick_params(axis='x', labelrotation=10)

Step 2-4. Point Plot

In [ ]:
### POINT PLOT : 범주형 데이터에 따른 수치형 데이터 평군과 신뢰구간 표시
# m행 n열 Figure 준비
mpl.rc('font', size=11)
figure, axes = plt.subplots(nrows=5) # 5행 1열 Figure 생성
figure.set_size_inches(12, 18) # 전체 Figure 크기 설정

# 각 서브플롯 할당
sns.pointplot(x='hour', y='count', data=train, hue='workingday', ax=axes[0])
sns.pointplot(x='hour', y='count', data=train, hue='holiday', ax=axes[1])
sns.pointplot(x='hour', y='count', data=train, hue='weekday', ax=axes[2])
sns.pointplot(x='hour', y='count', data=train, hue='season', ax=axes[3])
sns.pointplot(x='hour', y='count', data=train, hue='weather', ax=axes[4])   # ★ weather==4 이상치 제거

Step 2-5. Scatter Plot


In [ ]:
### SCATTER PLOT : 수치형 데이터 상관관계 파악
# Figure 준비
mpl.rc('font', size=15)
figure, axes = plt.subplots(nrows=2, ncols=2) # Figure 생성
plt.tight_layout()
figure.set_size_inches(7, 6)       # 전체 Figure 크기 설정

# 서브플롯 할당
sns.regplot(x='temp', y='count', data=train, ax=axes[0, 0],
            scatter_kws={'alpha': 0.2}, line_kws={'color': 'blue'})
sns.regplot(x='atemp', y='count', data=train, ax=axes[0, 1],
            scatter_kws={'alpha': 0.2}, line_kws={'color': 'blue'})
sns.regplot(x='windspeed', y='count', data=train, ax=axes[1, 0],
            scatter_kws={'alpha': 0.2}, line_kws={'color': 'blue'}) # ★ 결측값 많음 → windspeed 피처 삭제
sns.regplot(x='humidity', y='count', data=train, ax=axes[1, 1],
            scatter_kws={'alpha': 0.2}, line_kws={'color': 'blue'})

Step 2-6. Heatmap

In [ ]:
### Heatemap : 수치형 데이터 상관관계 파악
train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr() # 수치형 데이터 간 상관관계 메트릭스

In [ ]:
corrMat = train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()
fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
sns.heatmap(corrMat, annot=True)  # 상관관계 히트맵 그리기
ax.set(title='Heatmap of Numerical Data') # ★ windspeed는 count와 상관관계 희미

Step 2-7. ★ 분석결과

* [Traget] count: 비대칭 분포로 학습 수행시 log 변환 수행(회귀모델 정규분포일수록 성능 좋음), 마지막에는 지수 변환하여 실제 타깃값 복원 필요
* weather : weater==4 이상치 발생 → 이상치 제거
* windspeed : 결측값 많음, count와 상관관계 없음 → 제거
* day, minute, seconde : 분별력 없음 → 제거

# STEP 3. Feature Engineering

Step 3-1. Load Data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content//'

# 훈련, 검증, 테스트 데이터 경로 설정
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

Step 3-2. Remove Outlier

In [ ]:
# 훈련 데이터에서 weather = 4 아닌 데이터만 추출
train = train[train['weather'] != 4]

Step 3-3. Concat Data(Apply same feature engineering with train, test)

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data.tail(3)

Step 3-4. Add Feature

In [ ]:
from datetime import datetime

# 날짜 피처 생성(요일 생성을 위해서)
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
# 연도 피처 생성
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
# 월 피처 생성
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
# 시 피처 생성
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])
# 요일 피처 생성
all_data['weekday'] = all_data['date'].apply(lambda dateString : datetime.strptime(dateString, "%Y-%m-%d").weekday())

In [ ]:
all_data.head(2)

Step 3-5. Remove Feature

In [ ]:
drop_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month']
# season 피처가 month 대분류 성격이라 month 불필요

all_data = all_data.drop(drop_features, axis=1)

In [ ]:
all_data.tail(2)

Step 3-6. Divide Data(train, test)

In [ ]:
# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])] # count null 아니면 train
X_test = all_data[pd.isnull(all_data['count'])]   # count null 이면 test

# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count'] # 타깃값

X_train.shape, X_test.shape

In [ ]:
X_train.head()

# STEP 4. Model

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

# STEP 5. Learning

Step 5-1. Setting

In [ ]:
# RMSLE
import numpy as np

def rmsle(y_true, y_pred, converExp=True):
  # 지수 변환
  if converExp:
    y_true = np.exp(y_true)
    y_pred = np.exp(y_pred)

  # 로그 변화 후 결측값을 0으로 변환
  log_true = np.nan_to_num(np.log(y_true+1))
  log_pred = np.nan_to_num(np.log(y_pred+1))

  # RMSLE 계산
  output = np.sqrt(np.mean(log_true - log_pred)**2)
  return output

# 타깃값 로그 전환(분석 정리)
log_y = np.log(y)

Step 5-2. Learning

In [ ]:
linear_reg_model.fit(X_train, log_y)  # 모델 훈련

# STEP 6. Validation

In [ ]:
preds = linear_reg_model.predict(X_train) # 검증 데이터로 해야함 다음 장부터 제대로 구현

In [ ]:
print(f"선형 회귀 RMSLE 값: {rmsle(log_y, preds, True):.4f}")

# STEP 7. Prediction and Submission

Step 7-1. Prediction

In [ ]:
preds = linear_reg_model.predict(X_test) # 테스트 데이터로 예측

Step 7-5. Submission

In [ ]:
submission['count'] = np.exp(preds)   # 지수 변환
submission.to_csv('submission.csv', index=False)                       # 제출 파일 생성